In [1]:
# 必要なライブラリをインポート
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

##Tokenizer, 事前学習されたLanguage modelを準備，日本語BERTを指定
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
model = AutoModelForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
##Language modelを推論モードに
model.eval()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [2]:
##一部がマスクされた文章を用意．マスクは[MASK]で表現．今回は２つ
text = 'ある日の[MASK]の事である。一人の下人が、羅生門の下で[MASK]を待っていた。'
##文章をBERT内で扱われる数値表現に変換
input_ids = tokenizer.encode(text, return_tensors='pt')
##[MASK]の位置をリストで取得しておく．
masked_indices = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()

with torch.no_grad():##計算グラフを作らず勾配計算を行わないようにする．
    results = model(input_ids)##[MASK]部分をまとめて推論
    for masked_index in masked_indices:##[MASK]の位置ごとに
        print("Position:" + str(masked_index))
        
        predictions = results[0][0, masked_index].topk(5)# 推論結果上位5件を取得
        pred_ids = predictions.indices.tolist()# 結果をリスト形式に
    
        for i, pred_id in zip(range(len(pred_ids)), pred_ids):##1位から順に表示
            word_ids = [pred_id]
            decoded_word = tokenizer.decode(word_ids)## 数値を実際の単語に変換
            print(str(i+1) + ':' + decoded_word)
    
    

Position:4
1:夜
2:朝
3:晩
4:夕方
5:深夜
Position:23
1:彼
2:帰り
3:夜
4:[UNK]
5:旅人
